In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import collections

# Model

In [ ]:
def create_diag_u(u, m):
#    print(u)
    reverse_u = [u[-1 * i] for i in range(1, len(u)+1)]
    mat = np.zeros((m,m))
    for i in range(m):
        if i <= m - len(u):
            mat[i,i:i+len(u)] = reverse_u
        else:
            mat[i,i:] = reverse_u[:m-i]
        #print(mat)
    
    return mat

In [ ]:
def add_vi_CSVS(model, y, m, x, z, A, B, prt = False):
    print('Adding Valida Inequalities on Counting')
    u = [sum(y[:i+1]) for i in range(len(y))]
    v = [sum(y[i:]) for i in range(len(y))]
    if prt:
        print('y:', y)
        print('u:', u)

    zeros = [0 for i in range( m - len(y) )]
    u_diag = create_diag_u(u, m)
    v_tilde = v + zeros

    if prt:
        print('u_diag:', u_diag)
        print('v_tilde:', v_tilde)
        print()


    #equation 2
    model.addConstrs((gp.quicksum( [A[i,j] * x[j] for j in range(m)]) >= 
                      gp.quicksum([u_diag[i,j] * z[j] for j in range(m)]) - 
                      gp.quicksum([u_diag[i+1,j] * z[j] for j in range(m)])
                      for i in range(m-1)) ,'two')
    model.addConstr((gp.quicksum( [A[(m-1),j] * x[j] for j in range(m)]) >= gp.quicksum([u_diag[m-1,j] * z[j] for j in range(m)])) ,'two2')

    #equation 3
    model.addConstrs((gp.quicksum( [B[i,j] * x[j] for j in range(m)]) >= v_tilde[i] for i in range(m)) ,'three')
 
    return model

In [ ]:
def add_vi_CS(model, y, m, x, z, A, B, prt = False):
    print('Adding Valida Inequalities on Counting')
    u = [sum(y[:i+1]) for i in range(len(y))]
    v = [sum(y[i:]) for i in range(len(y))]

    if prt:
        print('y:', y)
        print('u:', u)
        print('v:', v)

    zeros = [0 for i in range( m - len(y) )]
    u_tilde = zeros + u
    v_tilde = v + zeros
    
    if prt:
        print('u_diag:', u_tilde)
        print('v_tilde:', v_tilde)
        print()


    #equation 2
    model.addConstrs((gp.quicksum( [A[i,j] * x[j] for j in range(m)]) >= u_tilde[i] for i in range(m)) ,'two')

    #equation 3
    model.addConstrs((gp.quicksum( [B[i,j] * x[j] for j in range(m)]) >= v_tilde[i] for i in range(m)) ,'three')
 
    return model

In [ ]:
def build_model(y_set, valid_inequality_CS = False,valid_inequality_CSVS = False, prt = True, m_huristic = -1):
    max_value = max([max(y) for y in y_set])
    
    #maximum length
    m = sum([len(i) for i in y_set]) 
    #set m if huristic is passed
    if m_huristic >0:
        m = min(m, m_huristic)
    
    if prt:
        print('Maximum_length:', max_length)
        print()
        
    #model
    model = gp.Model('model')
    
    #length Variable
    z = model.addVars(range(m), vtype = GRB.BINARY, obj = [1 for i in range(m)], name = 'z')
    x = model.addVars(range(m), vtype = GRB.INTEGER, lb = 0, name = 'x')
    
    #equation 6
    model.addConstrs((x[i] <= z[i] * max_value for i in range(m)), 'six')
    #equation 7
    model.addConstrs((z[i] >= z[i+1] for i in range(m-1)), 'seven')
    
    #upper triangular mat
    A = np.tril( np.ones((m,m)) )
    #lower triangular mat
    B = np.triu(np.ones((m,m)) ) 
    if prt:
        print('A = \n', A)
        print('B = \n', B)
        print()
        
        
    c_array = []
    d_array = []
    for y in y_set: 
        
        if valid_inequality_CSVS or valid_inequality_CS:
            print('Adding Valida Inequalities on Counting')
            u = [sum(y[:i+1]) for i in range(len(y))]
            v = [sum(y[i:]) for i in range(len(y))]

            zeros = [0 for i in range( m - len(y) )]
            v_tilde = v + zeros
            model.addConstrs((gp.quicksum( [B[i,j] * x[j] for j in range(m)]) >= v_tilde[i] for i in range(m)) ,'three')
 
            
            if valid_inequality_CSVS:
                u_diag = create_diag_u(u, m)
                model.addConstrs((gp.quicksum( [A[i,j] * x[j] for j in range(m)]) >= 
                      gp.quicksum([u_diag[i,j] * z[j] for j in range(m)]) - 
                      gp.quicksum([u_diag[i+1,j] * z[j] for j in range(m)])
                      for i in range(m-1)) ,'two')
                model.addConstr((gp.quicksum( [A[(m-1),j] * x[j] for j in range(m)]) >=
                                 gp.quicksum([u_diag[m-1,j] * z[j] for j in range(m)])) ,'two2')

            elif valid_inequality_CS:
                u_tilde = zeros + u

                #equation 2
                model.addConstrs((gp.quicksum( [A[i,j] * x[j] for j in range(m)]) >= u_tilde[i] for i in range(m)) ,'two')

        
    
        #covering constraint
        c = model.addVars(m, len(y), vtype = GRB.BINARY)
        d = model.addVars(m, vtype = GRB.INTEGER, lb = 0)
        model.addConstrs((gp.quicksum([c[i,j] * y[j] for j in range(len(y))]) + d[i] == x[i] for i in range(m)), 'cover')
        model.addConstrs((gp.quicksum([c[i,j] for j in range(len(y))])*max_value + d[i] <= max_value for i in range(m)), 'full')
        model.addConstrs((gp.quicksum([c[i,j] for i in range(m)]) == 1 for j in range(len(y))), 'full2')
        m_index = [m-i for i in range(m)]
        model.addConstrs((gp.quicksum([c[i,j] * m_index[i] for i in range(m)]) >= 
                         gp.quicksum([c[i,j+1] * m_index[i] for i in range(m)]) for j in range(len(y) - 1)), 'sequential')
        
        c_array.append(c)
        d_array.append(d)
        
    if prt:
        print("total_flips:", total_flips)
        
    var = {}
        
 
    var['x'] = x
    var['c_array'] = c_array
    var['d_array'] = d_array
    var['z'] = z
    return model, var
    

In [ ]:
def BitMajorityVoting(y_set):
    result = []
    
    while sum([len(y) for y in y_set]) > 0:
        
        first_bits = [y[0] for y in y_set if len(y) > 0]
        vote = collections.Counter(first_bits).most_common(1)[0][0]
        result.append(vote)
        
        y_set_new = [y[1:] if len(y) > 0 and y[0] == vote else y for y in y_set]
        
        y_set = y_set_new
        
    return result


In [ ]:
def solve(subseq_set, m = -1, valid_inequality_CS = False, valid_inequality_CSVS = False, time_limit = -1):
    
    start_time = time.time()
    model,var = build_model(y_set, 
                valid_inequality_CS = valid_inequality_CS, 
                valid_inequality_CSVS = valid_inequality_CSVS, 
                prt = False, 
                m_huristic = m)
    if time_limit != -1:
        model.setParam('TimeLimit', time_limit)
        
    print('Build Time:', np.round( (time.time() - start_time) , 2) )
    model.optimize()
    cnt = model.SolCount
        
    if cnt == 0:
        print('No Solution Found for: m = ', m)
        return [], var

    
    print('############# Result ##################\n')
    x = var['x']
    z = var['z']
    seq = []
    for i in range(len(z)):
        if i == 0:
            seq.append(int(x[i].x))
        elif z[i].x > 0:
            seq.append(int(x[i].x))
            
    print('X:', (seq))
    print('length:', len(seq))
    
    return seq, var

# Testing

In [ ]:
#Data Generation
def generating_y_set_conti(x, missing = 1):
    y = [x[:i] + x[i+missing:] for i in range(len(x) - missing)]
    return y

def generating_y_set_partial(x, seq_count, missing):
    y_set_index = [np.sort(np.random.choice(len(x), size = len(x) - missing, replace = False)) for i in range(seq_count)]
    y_set = [np.array(x)[y] for y in y_set_index]
    return y_set


In [ ]:
time_limit = 100
sizes = 40
sample_rates = [i / 10 for i in range(1,11,3)]
sequence_counts = [i for i in range(2,50,15)]

IP_time = {}
IPCS_time = {}
IPCSVS_time = {}

IP_result = {}
IPCS_result = {}
IPCSVS_result = {}

for sample_rate in sample_rates:
    for sequence_count in sequence_counts:
        
        print('\n\nCurrent: ', (sample_rate, sequence_count))
        if (sample_rate, sequence_count) in IP_time:
            assert (sample_rate, sequence_count) in IP_time
            assert (sample_rate, sequence_count) in IPCS_time
            assert (sample_rate, sequence_count) in IPCSVS_time
            
            assert (sample_rate, sequence_count) in IP_result
            assert (sample_rate, sequence_count) in IPCS_result
            assert (sample_rate, sequence_count) in IPCSVS_result
            print('Pass')
            
            continue
            
            
        IP_time_arr = []
        IPCS_time_arr = []
        IPCSVS_time_arr = []
        
        IP_result_arr = []
        IPCS_result_arr = []
        IPCSVS_result_arr = []
        
        for _ in range(1):
            true_sequence = list(np.random.choice(range(100), size = size, replace = True))
            true_sequence[-1] = 1
            y_set = generating_y_set_partial(true_sequence, seq_count = sequence_count, missing = int(len(true_sequence) * (1-sample_rate)))
            m = len(BitMajorityVoting(y_set))

            print('m = ', m)
            ##########################################
            starting_time = time.time()
            seq,var = solve(y_set, m = m,valid_inequality_CS = False, valid_inequality_CSVS = False, time_limit = time_limit)
            ####### STATS #######
            if len(seq) == len(true_sequence) and sum([true_sequence[i] != seq[i] for i in range(len(true_sequence))]) == 0:
                    IP_result_arr.append(1)
            else:
                IP_result_arr.append(0)
            ####### TIME #######
            IPCS_time_arr.append(time.time() - starting_time)

            ##########################################
            starting_time = time.time()
            seq,var = solve(y_set, m = m,valid_inequality_CS = True, valid_inequality_CSVS = False, time_limit = time_limit)
            ####### STATS #######
            if len(seq) == len(true_sequence) and sum([true_sequence[i] != seq[i] for i in range(len(true_sequence))]) == 0:
                    IPCS_result_arr.append(1)
            else:
                IPCS_result_arr.append(0)
            ####### TIME #######
            IPCS_time_arr.append(time.time() - starting_time)
            
            ##########################################
            starting_time = time.time()
            seq,var = solve(y_set, m = m,valid_inequality_CS = False, valid_inequality_CSVS = True, time_limit = time_limit)
            ####### STATS #######
            if len(seq) == len(true_sequence) and sum([true_sequence[i] != seq[i] for i in range(len(true_sequence))]) == 0:
                    IPCSVS_result_arr.append(1)
            else:
                IPCSVS_result_arr.append(0)
            ####### TIME #######
            IPCSVS_time_arr.append(time.time() - starting_time)
        
        IP_time[(sample_rate, sequence_count)] = IP_time_arr
        IPCS_time [(sample_rate, sequence_count)] = IPCS_time_arr
        IPCSVS_time [(sample_rate, sequence_count)] = IPCSVS_time_arr
        
        
        IP_result[(sample_rate, sequence_count)] = IP_result_arr
        IPCS_result[(sample_rate, sequence_count)] = IPCS_result_arr
        IPCSVS_result[(sample_rate, sequence_count)] = IPCSVS_result_arr

        
        